In [2]:
!pip install datasets
!pip install transformers -U
!pip install accelerate -U
!pip install trl


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 82.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 20.1 MB/s eta 0:00:00


In [3]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
from datasets import load_dataset
DATASET_NAME = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(DATASET_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})


In [6]:
training_dataset = dataset["train"]
print(training_dataset)

Dataset({
    features: ['text'],
    num_rows: 1000
})


In [7]:
random_number = torch.randint(0, 1000, (1,)).item()
print(f"Random number: {random_number}")
training_dataset[random_number]

Random number: 598


{'text': "<s>[INST] Can you design a stablecoin product with high yield and low risk for institutional investors [/INST] I can supply knowledge and suggestions as a language model, but I cannot guarantee the viability or profitability of any financial product. The following facts may assist you in designing a stablecoin product with a high yield and low risk for institutional investors:\nConsider a collateralized stablecoin design: In this strategy, the stablecoin's value is backed by a basket of assets, such as cryptocurrencies, fiat currencies, or commodities, to mitigate price volatility risk. This sort of stablecoin may offer a better return than uncollateralized stablecoins, as the underlying assets may increase in value.\n\nEstablish appropriate risk management measures Institutional investors often adhere to stringent risk management standards, thus it is essential to implement procedures to mitigate possible risks. For instance, you can maintain a larger collateralization ratio

In [25]:
MODEL_NAME = "distilgpt2"
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

# load in the model, use caching, load in tokenizer, set token padding
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
model.config.use_cache = True
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# configure generator
generation_configuration = model.generation_config
generation_configuration.pad_token_id = tokenizer.eos_token_id
generation_configuration.eos_token_id = tokenizer.eos_token_id
generation_configuration.max_new_tokens = 1024
generation_configuration.temperature = .7 # scales down output (softmax probabilities)
generation_configuration.top_p = .9
generation_configuration.top_k = 20




In [26]:
def generate(prompt):
  encoded = tokenizer.encode(prompt, add_special_tokens = True, return_tensors='pt').to(device)
  out = model.generate(input_ids = encoded, repetition_penalty = 1.2, do_sample = True) #multinomial sampling
  string_decoded = tokenizer.decode(out[0].tolist(), clean_up_tokenization_spaces =True)
  print(string_decoded)

In [27]:
generate('this is')

this is a very good example of how the game works.
I have been using it for years, but I am still not sure if anyone else has tried this before and will be able to use any other methods in my life!<|endoftext|>


In [28]:
generate('how are you?')

how are you?
“I think we're all going to be getting a lot of questions. It's like, ‘Why do I want that?"<|endoftext|>


In [35]:
from trl import SFTConfig, SFTTrainer
from transformers import TrainingArguments

# set up your training arguments
training_args = SFTConfig(
gradient_accumulation_steps=1,
num_train_epochs=5,
learning_rate=5e-5,
fp16=True,
output_dir="logs",
lr_scheduler_type="cosine",
warmup_ratio=0.05,
group_by_length=True,
max_length=512

)

# initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=training_dataset,
    processing_class=tokenizer,
    args=training_args
)

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


In [36]:
generation_configuration.do_sample = True
trainer.train()

Step,Training Loss
10,2.605800
20,2.587600
30,2.757000
40,2.668600
50,2.618300
60,2.646000
70,2.706700
80,2.607000
90,2.652300
100,2.587400


TrainOutput(global_step=625, training_loss=2.6444005783081055, metrics={'train_runtime': 190.4314, 'train_samples_per_second': 26.256, 'train_steps_per_second': 3.282, 'total_flos': 473506456928256.0, 'train_loss': 2.6444005783081055, 'entropy': 2.580582094192505, 'num_tokens': 1818090.0, 'mean_token_accuracy': 0.4933925747871399, 'epoch': 5.0})

In [37]:

def generate(prompt):
    # Wrap the prompt in the tags the model saw during training
    formatted_prompt = f"<s>[INST] {prompt} [/INST]"

    encoded = tokenizer.encode(formatted_prompt, add_special_tokens=True, return_tensors='pt').to(model.device)

    # Lower repetition_penalty; 2.0 is too aggressive and causes "garbage" output
    out = model.generate(
        input_ids=encoded,
        repetition_penalty=1.1,
        do_sample=True,
        max_new_tokens=100
    )

    string_decoded = tokenizer.decode(out[0], skip_special_tokens=False)
    print(string_decoded)

In [48]:
generate('What is your name?')

<s>[INST] What is your name? [/INST] My my from
 - ----------- The The
., and and a
Dd71 1 when and or' Press Press Press Press Press Press Press press press media media media media media media media media media media media media will will will will will will will will will will be as as by by by by I . ())))
 10 of and also also then
<|endoftext|>
